In [1]:
import pandas as pd
import numpy as np

## 1) Only when using ExpoApp data: 
## Convert ExpoApp data to equidistant time series 

### 1.1) Scenario 1: Further analyses will be performed with the pa_proc pipeline ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_equidistant

# Create dataframe with equidistant acceleration in three axes
data_eq = expoapp_to_equidistant("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv", 1, "Europe/Madrid")

# Save dataframe to csv (optional)
data_eq.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", index=False, header=False)

### 1.2) Scenario 2: Further analyses will be performed with GGIR ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_actigraph

# Create dataframe with equidistant acceleration in three axes, resembling an Actigraph file (to be used in GGIR)
data_act = expoapp_to_actigraph("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv",  1, "Europe/Madrid")

# Save dataframe to csv (optional)
data_act.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_act.csv", index=False, header=False)

## 2) Convert raw acceleration data to Actigraph counts and ENMO

### 2.1) Scenario 1: Equidistant ExpoApp data in .CSV format

In [ ]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", 
                                          'expoapp-csv', 30, 1)

### 2.2) Scenario 2: Actigraph data in .CSV format

In [4]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB033 (2022-08-08)RAW.csv",
                                          'actigraph-csv', 30, 1)

### 2.3) Scenario 3: Actigraph data in .gt3x format

In [2]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB_033 (2022-08-08).gt3x",
                                          'actigraph-gt3x', 30, 1)

## 3) calculate physical activity variables

In [8]:
# Identify different days in data
days_measured = intermediate_60s.index.map(pd.Timestamp.date).unique()
 
# Create results dataframe
results = pd.DataFrame(index=days_measured)
 
# Calculate time in different PA domains (cut-offs used from document Gabriela)
for day_loop in days_measured:
	
	day_data = intermediate_60s[(intermediate_60s.index.date == day_loop)]
	
	results.loc[day_loop, 'weartime_choi'] = np.sum(day_data.wear_choi == 1)
	results.loc[day_loop, 'weartime_vanhees'] = np.sum(day_data.wear_vanhees == 1)
	
	day_data_wear_choi = day_data[day_data.wear_choi == 1]
	day_data_wear_vanhees = day_data[day_data.wear_vanhees == 1]
	
	results.loc[day_loop, 'counts_lpa'] = np.sum((day_data_wear_choi.counts_vm > 200) & (day_data_wear_choi.counts_vm <= 2690))
	results.loc[day_loop, 'counts_mvpa'] = np.sum(day_data_wear_choi.counts_vm > 2690)
	results.loc[day_loop, 'counts_vpa'] = np.sum(day_data_wear_choi.counts_vm > 6166)
 
 
	results.loc[day_loop, 'enmo_lpa'] = np.sum((day_data_wear_vanhees.enmo > 47.4) & (day_data_wear_vanhees.enmo < 69.1))
	results.loc[day_loop, 'enmo_mvpa'] = np.sum(day_data_wear_vanhees.enmo >= 69.1)
	results.loc[day_loop, 'enmo_vpa'] = np.sum(day_data_wear_vanhees.enmo >= 258.7)

results

,weartime_choi,weartime_vanhees,counts_lpa,counts_mvpa,counts_vpa,enmo_lpa,enmo_mvpa,enmo_vpa
2022-08-08,476.0,480.0,186.0,43.0,13.0,15.0,45.0,4.0
2022-08-09,559.0,615.0,240.0,40.0,7.0,9.0,43.0,5.0
2022-08-10,503.0,660.0,139.0,37.0,8.0,17.0,31.0,2.0
2022-08-11,501.0,510.0,165.0,2.0,0.0,12.0,0.0,0.0
2022-08-12,616.0,675.0,176.0,83.0,17.0,16.0,81.0,3.0
2022-08-13,410.0,465.0,150.0,10.0,0.0,7.0,12.0,0.0
2022-08-14,698.0,735.0,149.0,5.0,0.0,5.0,7.0,0.0
2022-08-15,723.0,765.0,123.0,2.0,0.0,2.0,0.0,0.0
2022-08-16,760.0,478.0,68.0,1.0,0.0,1.0,0.0,0.0
